In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

# Import your reusable feature function
from modules.feature_engineering.feature_utils import make_features


In [ ]:
train = pd.read_csv("../baseline_prophet_forecast/data/train.csv")
features = pd.read_csv("../baseline_prophet_forecast/data/features.csv")
stores = pd.read_csv("../baseline_prophet_forecast/data/stores.csv")

df = (
    train
    .merge(features, on=["Store", "Date", "IsHoliday"])
    .merge(stores, on="Store")
)

df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Store", "Dept", "Date"])


In [ ]:
#Select Store–Dept (for demo) and build weekly series
store = 1
dept = 1

ts = df[(df["Store"] == store) & (df["Dept"] == dept)][["Date", "Weekly_Sales", "IsHoliday"]].copy()
ts = ts.rename(columns={"Date": "ds", "Weekly_Sales": "y"}).sort_values("ds")

#ts = ts.set_index("ds").asfreq("W")
ts.head()
ts = ts.rename(columns={"y": "Weekly_Sales"})


In [ ]:
#Generate features
feat = make_features(ts, target="Weekly_Sales").dropna()

target_col = "Weekly_Sales"
feature_cols = [c for c in feat.columns if c != target_col]

X = feat[feature_cols]
y = feat[target_col]


target_col = "Weekly_Sales"
feature_cols = [c for c in feat.columns if c != target_col]

X = feat[feature_cols]
y = feat[target_col]


In [ ]:
#Time-based train/validation split (last 12 weeks as holdout)
h = 12

X_train, X_val = X.iloc[:-h], X.iloc[-h:]
y_train, y_val = y.iloc[:-h], y.iloc[-h:]

val_index = y_val.index


In [ ]:
#Train LightGBM
params = dict(
    objective="regression",
    learning_rate=0.05,
    n_estimators=600,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model = lgb.LGBMRegressor(**params)
model.fit(X_train, y_train)


In [ ]:
# Predict
pred = model.predict(X_val)

mae = mean_absolute_error(y_val, pred)
wape = np.sum(np.abs(y_val - pred)) / np.sum(np.abs(y_val)) * 100

print(f"MAE:  {mae:,.2f}")
print(f"WAPE: {wape:,.2f}%")


In [ ]:
#Plot Forecast vs Actual
plt.figure(figsize=(12, 5))
plt.plot(val_index, y_val.values, marker="o", label="Actual")
plt.plot(val_index, pred, marker="x", label="Forecast")
plt.title(f"LightGBM Forecast vs Actual — Store {store}, Dept {dept}")
plt.xlabel("Date")
plt.ylabel("Weekly Sales")
plt.legend()
plt.tight_layout()
plt.savefig("images/forecast_vs_actual_store1_dept1.png")
plt.show()


In [ ]:
# Feature Importance
importances = pd.Series(model.feature_importances_, index=feature_cols).sort_values(ascending=False).head(20)

plt.figure(figsize=(10, 6))
importances.sort_values().plot(kind="barh")
plt.title("Top 20 Feature Importances (LightGBM)")
plt.tight_layout()
plt.savefig("images/feature_importance_top20.png")
plt.show()


In [ ]:
import joblib
joblib.dump(model, "lgbm_model_store1_dept1.pkl")


#Inferences
Global model strategy: In production, this becomes a single LightGBM model trained across many Store–Dept/SKU series using identifiers and metadata as features.

Batch inference: Forecasts run nightly for all entities and write to a warehouse table consumed by dashboards.

Monitoring: Track WAPE by store/dept cluster; alert on systematic under-forecasting (stock-out risk) and over-forecasting (overstock risk).